# Generalized Shallow Water Equations

In [27]:
# Primitive Variables
h = var('h', domain='positive')
u = var('u')
s = var('s')
k = var('k')
m = var('m')
g = var('g', domain='positive')

# conserved variables
q1 = var('q_1', domain='positive')
q2 = var('q_2')
q3 = var('q_3')
q4 = var('q_4')
q5 = var('q_5')

# dictionaries to substitute between primitve and conserved and vice versa
conserved_to_primitive = {q1:h, q2: h*u, q3: h*s, q4: h*k, q5: h*m}
primitive_to_conserved = {h:q1, u: q2/q1, s: q3/q1, k: q4/q1, m: q5/q1}

## First Order/Linear System

In [25]:
flux_primitive = vector([h*u, h*u^2 + 1/2 * g * h^2 + 1/3 * h * s^2, 2*h*u*s])
Q_primitive = matrix([[0, 0, 0],[0, 0, 0],[0, 0, u]])

flux_conserved = flux_primitive.subs(primitive_to_conserved)
print(flux_conserved)
Q_conserved = Q_primitive.subs(primitive_to_conserved)

(q_2, 1/2*g*q_1^2 + q_2^2/q_1 + 1/3*q_3^2/q_1, 2*q_2*q_3/q_1)


In [4]:
jacobian_conserved = jacobian(flux_conserved, (q1, q2, q3))
print(jacobian_conserved)

[                                    0                                     1                                     0]
[g*q_1 - q_2^2/q_1^2 - 1/3*q_3^2/q_1^2                             2*q_2/q_1                           2/3*q_3/q_1]
[                     -2*q_2*q_3/q_1^2                             2*q_3/q_1                             2*q_2/q_1]


In [5]:
jacobian_primitive = jacobian_conserved.subs(conserved_to_primitive)
print(jacobian_primitive)

[                  0                   1                   0]
[g*h - 1/3*s^2 - u^2                 2*u               2/3*s]
[             -2*s*u                 2*s                 2*u]


In [19]:
quasilinear_matrix_primitive = jacobian_primitive - Q_primitive
print(quasilinear_matrix_primitive)

[                  0                   1                   0]
[g*h - 1/3*s^2 - u^2                 2*u               2/3*s]
[             -2*s*u                 2*s                   u]


In [20]:
quasilinear_matrix_conserved = jacobian_conserved - Q_conserved
print(quasilinear_matrix_conserved)

[                                    0                                     1                                     0]
[g*q_1 - q_2^2/q_1^2 - 1/3*q_3^2/q_1^2                             2*q_2/q_1                           2/3*q_3/q_1]
[                     -2*q_2*q_3/q_1^2                             2*q_3/q_1                               q_2/q_1]


In [21]:
eigenvalues_primitive = quasilinear_matrix_primitive.eigenvalues()
print([e.full_simplify() for e in eigenvalues_primitive])

[u - sqrt(g*h + s^2), u + sqrt(g*h + s^2), u]


## Second Order/Quadratic

In [28]:
flux_primitive = vector([h*u, 
                         h*u^2 + 1/2 * g * h^2 + 1/3 * h * s^2 + 1/5 * h * k^2, 
                         2*h*u*s + 4/5*h*s*k, 
                         2*h*u*k + 2/3*h*s^2+2/7*h*k^2])
Q_primitive = matrix([[0, 0, 0, 0],
                      [0, 0, 0, 0],
                      [0, 0, u-k/5, s/5],
                      [0, 0, s, u+k/7]])

flux_conserved = flux_primitive.subs(primitive_to_conserved)
print(flux_conserved)
Q_conserved = Q_primitive.subs(primitive_to_conserved)

(q_2, 1/2*g*q_1^2 + q_2^2/q_1 + 1/3*q_3^2/q_1 + 1/5*q_4^2/q_1, 2*q_2*q_3/q_1 + 4/5*q_3*q_4/q_1, 2/3*q_3^2/q_1 + 2*q_2*q_4/q_1 + 2/7*q_4^2/q_1)


In [29]:
jacobian_conserved = jacobian(flux_conserved, (q1, q2, q3, q4))
print(jacobian_conserved)

[                                                      0                                                       1                                                       0                                                       0]
[g*q_1 - q_2^2/q_1^2 - 1/3*q_3^2/q_1^2 - 1/5*q_4^2/q_1^2                                               2*q_2/q_1                                             2/3*q_3/q_1                                             2/5*q_4/q_1]
[                   -2*q_2*q_3/q_1^2 - 4/5*q_3*q_4/q_1^2                                               2*q_3/q_1                                 2*q_2/q_1 + 4/5*q_4/q_1                                             4/5*q_3/q_1]
[   -2/3*q_3^2/q_1^2 - 2*q_2*q_4/q_1^2 - 2/7*q_4^2/q_1^2                                               2*q_4/q_1                                             4/3*q_3/q_1                                 2*q_2/q_1 + 4/7*q_4/q_1]


In [37]:
jacobian_primitive = jacobian_conserved.subs(conserved_to_primitive)
print(jacobian_primitive[0])
print(jacobian_primitive[1])
print(jacobian_primitive[2])
print(jacobian_primitive[3])

(0, 1, 0, 0)
(g*h - 1/5*k^2 - 1/3*s^2 - u^2, 2*u, 2/3*s, 2/5*k)
(-4/5*k*s - 2*s*u, 2*s, 4/5*k + 2*u, 4/5*s)
(-2/7*k^2 - 2/3*s^2 - 2*k*u, 2*k, 4/3*s, 4/7*k + 2*u)


In [38]:
quasilinear_matrix_primitive = jacobian_primitive - Q_primitive
print(quasilinear_matrix_primitive[0])
print(quasilinear_matrix_primitive[1])
print(quasilinear_matrix_primitive[2])
print(quasilinear_matrix_primitive[3])

(0, 1, 0, 0)
(g*h - 1/5*k^2 - 1/3*s^2 - u^2, 2*u, 2/3*s, 2/5*k)
(-4/5*k*s - 2*s*u, 2*s, k + u, 3/5*s)
(-2/7*k^2 - 2/3*s^2 - 2*k*u, 2*k, 1/3*s, 3/7*k + u)


In [32]:
quasilinear_matrix_conserved = jacobian_conserved - Q_conserved
print(quasilinear_matrix_conserved)

[                                                      0                                                       1                                                       0                                                       0]
[g*q_1 - q_2^2/q_1^2 - 1/3*q_3^2/q_1^2 - 1/5*q_4^2/q_1^2                                               2*q_2/q_1                                             2/3*q_3/q_1                                             2/5*q_4/q_1]
[                   -2*q_2*q_3/q_1^2 - 4/5*q_3*q_4/q_1^2                                               2*q_3/q_1                                       q_2/q_1 + q_4/q_1                                             3/5*q_3/q_1]
[   -2/3*q_3^2/q_1^2 - 2*q_2*q_4/q_1^2 - 2/7*q_4^2/q_1^2                                               2*q_4/q_1                                             1/3*q_3/q_1                                   q_2/q_1 + 3/7*q_4/q_1]


In [33]:
eigenvalues_primitive = quasilinear_matrix_primitive.eigenvalues()
print([e.full_simplify() for e in eigenvalues_primitive])

[5/14*k + u - 1/210*sqrt(15)*sqrt((8575*g^2*h^2 + 11340*g*h*k^2 + 20412*k^4 + 32928*s^4 + 84*(490*g*h - 243*k^2)*s^2 + (-42875*g^3*h^3 + 103950*g^2*h^2*k^2 + 131220*g*h*k^4 + 157464*k^6 + 296352*s^6 + 504*(980*g*h + 27*k^2)*s^4 + 18*(8575*g^2*h^2 - 85050*g*h*k^2 - 34992*k^4)*s^2 + 90*sqrt(-6002500/3*g^5*h^5*k^2 - 2866500*g^4*h^4*k^4 - 3110400*g^3*h^3*k^6 - 524880*g^2*h^2*k^8 - 30118144/15*s^12 - 307328/5*(196*g*h - 405*k^2)*s^10 - 784/15*(552230*g^2*h^2 + 1222452*g*h*k^2 + 1622025*k^4)*s^8 - 8/15*(64706950*g^3*h^3 + 303714495*g^2*h^2*k^2 + 251547282*g*h*k^4 - 78928830*k^6)*s^6 - 1/5*(102942875*g^4*h^4 + 289560600*g^3*h^3*k^2 - 1220229360*g^2*h^2*k^4 - 1366017696*g*h*k^6 - 127545840*k^8)*s^4 - 1/15*(73530625*g^5*h^5 - 197182125*g^4*h^4*k^2 + 669173400*g^3*h^3*k^4 + 1046873160*g^2*h^2*k^6 + 1323642384*g*h*k^8 + 229582512*k^10)*s^2))^(1/3)*(490*g*h + 459*k^2 + 588*s^2) + 7*(-42875*g^3*h^3 + 103950*g^2*h^2*k^2 + 131220*g*h*k^4 + 157464*k^6 + 296352*s^6 + 504*(980*g*h + 27*k^2)*s^4 + 18*(85

In [40]:
a = var('a')
A = quasilinear_matrix_primitive - a * matrix.identity(SR, 4)
print(A)

[                           -a                             1                             0                             0]
[g*h - 1/5*k^2 - 1/3*s^2 - u^2                      -a + 2*u                         2/3*s                         2/5*k]
[             -4/5*k*s - 2*s*u                           2*s                    -a + k + u                         3/5*s]
[   -2/7*k^2 - 2/3*s^2 - 2*k*u                           2*k                         1/3*s                -a + 3/7*k + u]


In [42]:
d = A.det()
print(d)

a^4 - a^2*g*h - 10/7*a^3*k + 10/7*a*g*h*k - 6/35*a^2*k^2 - 3/7*g*h*k^2 + 22/35*a*k^3 - 1/35*k^4 - 6/5*a^2*s^2 + 1/5*g*h*s^2 - 6/35*a*k*s^2 - 6/35*k^2*s^2 + 1/5*s^4 - 4*a^3*u + 2*a*g*h*u + 30/7*a^2*k*u - 10/7*g*h*k*u + 12/35*a*k^2*u - 22/35*k^3*u + 12/5*a*s^2*u + 6/35*k*s^2*u + 6*a^2*u^2 - g*h*u^2 - 30/7*a*k*u^2 - 6/35*k^2*u^2 - 6/5*s^2*u^2 - 4*a*u^3 + 10/7*k*u^3 + u^4


In [43]:
d.factor()

a^4 - a^2*g*h - 10/7*a^3*k + 10/7*a*g*h*k - 6/35*a^2*k^2 - 3/7*g*h*k^2 + 22/35*a*k^3 - 1/35*k^4 - 6/5*a^2*s^2 + 1/5*g*h*s^2 - 6/35*a*k*s^2 - 6/35*k^2*s^2 + 1/5*s^4 - 4*a^3*u + 2*a*g*h*u + 30/7*a^2*k*u - 10/7*g*h*k*u + 12/35*a*k^2*u - 22/35*k^3*u + 12/5*a*s^2*u + 6/35*k*s^2*u + 6*a^2*u^2 - g*h*u^2 - 30/7*a*k*u^2 - 6/35*k^2*u^2 - 6/5*s^2*u^2 - 4*a*u^3 + 10/7*k*u^3 + u^4